# Batch Processing!
#### A notebook to show some of the capilities available through the pCunch package

This is certainly not an exhaustive look at everything that the pCrunch module can do, but should hopefully provide some insight. 
...or, maybe I'm just procrastinating doing more useful work.

In [ ]:
# Python Modules and instantiation
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import time
import os
# %matplotlib widget
# ROSCO toolbox modules 
from ROSCO_toolbox import utilities as rosco_utilities
# WISDEM modules
from wisdem.aeroelasticse.Util import FileTools
# Batch Analysis tools
from pCrunch import Processing, Analysis
from pCrunch import pdTools

# Instantiate fast_IO
fast_io = rosco_utilities.FAST_IO()
fast_pl = rosco_utilities.FAST_Plots()

import importlib
Processing = importlib.reload(Processing)
Analysis = importlib.reload(Analysis)

### Define file paths and filenames
I'm loading a case matrix output from using wisdem.aeroelasticse.CaseGen_General to run a series of batch runs to initialize the output files here. I'm just using some saved runs on my computer that are of no actual significants (and actually don't look great, but get the point accross).

In [ ]:
# point to some file paths
outfile_base = '../BatchOutputs/NREL5MW/5MW_Land/'
fname_case_matrix = '../BatchOutputs/NREL5MW/5MW_Land/case_matrix.yaml'

In [ ]:
# Load case matrix into datafraome
case_matrix = FileTools.load_yaml(fname_case_matrix, package=1)
cm = pd.DataFrame(case_matrix)
# cm[cm[('IEC','DLC')]=='1.1']
_, _, _, cmw = Processing.get_windspeeds(cm, return_df=True)
cmw.head()

In [ ]:
# Define DLCs we care to separate things by
DLCs = [1.1, 1.3]

In [ ]:
# Pare down case matrix for desired runs and load the outfiles
cm2 = pd.concat([cm[cm[('IEC','DLC')]==dlc] for dlc in DLCs]).reset_index()

# Parse find outfiles names
outfiles = []
for dlc in DLCs:
    case_names = cm2[cm2[('IEC','DLC')]==dlc]['Case_Name']
    outnames = list( outfile_base + case_names + '.outb' )
    outfiles.append(outnames)

In [ ]:
#### `outfiles`
In the end, we just need a list of OpenFAST output files. Here, we have a structure that looks something like `[[], []]`. This could be extended any amount like `[[],[],...,[], []]`, or just be one list of strings `[]`.

In [ ]:
outfiles

### Now we can do some processing!

First, let's load the FAST_Processing class and initialize some parameters.


In [ ]:
fp = Processing.FAST_Processing()
fp.OpenFAST_outfile_list = outfiles
fp.dataset_names = ['DLC1.1', 'DLC1.3']
fp.to = 30
fp.parallel_analysis = True
fp.save_LoadRanking = True
fp.save_SummaryStats = True
fp.verbose=True
# fp.ranking_vars = [["RotSpeed"], 
#                     ["OoPDefl1", "OoPDefl2", "OoPDefl3"], 
#                     ['RootMxc1', 'RootMxc2', 'RootMxc3'],
#                     ['TwrBsFyt'],
#                     ] 

#### The fast way to compare things.
We could now collect all of the summary stats and load rankings using:
```
stats,load_rankings = fp.batch_processing()
```
In `fp.batch_processing()` most of the analysis is done for any structure of data. I'm going to step through things a bit more piecewise in this notebook, however.

In [ ]:
# stats,load_rankings = fp.batch_processing()

### Design Comparisons
We can use fp.design_comparison to compare multiple sets of runs (like we are in this case...). This will generate summary stats and load rankings, running parralelizing when it can and is told to. `fp.batch_processing()` functionally does the same thing. We'll the design comparison here to show some speed-related results

In [ ]:
stats, load_ranking = fp.design_comparison(outfiles)

#### Breaking it down further...

`fp.batch_processing()` calls `Analysis.Loads_Analysls.full_loads_analysis()` to load openfast data, generate stats, and calculate load rankings. Because we defined `fp.parallel_analysis=True` this process was parallelized. This helps for speed and memory reasons, because now every openfast run is not saved. `fp.batch_processing()` then takes all of the output data and parses it back together. 

Separately, we call call `Analysis.Loads_Analysls.full_loads_analysis()` with `return_FastData=True` and all of the fast data will be returned. Because we are comparing data though, we'll stick with the design comparison tools.

### We can look at our data a bit further with pandas dataframes
The data here is just for a few runs for simplicity. Usually you'd do this for a LOT more cases...

In [ ]:
stats_df = pdTools.dict2df(stats, names=['DLC_1.1', 'DLC_1.3'])
stats_df

### Load Ranking
Lets re-run the load ranking for the sake of example. We'll have to load the analysis tools, and then run the load ranking for the stats we just 

In [ ]:
fa = Analysis.Loads_Analysis()
fa.t0 = 30
fa.verbose = False

Define the ranking variables and statiscits of interest. Note that `len(ranking_vars) == len(ranking_stats)`! We can pass this a list of stats (multiple runs), a dictionary with one run of stats, or a pandas dataframe with the requisite stats. We'll also output a dictionary and a pandas DataFrame from `fa.load_ranking()`

In [ ]:
fa.ranking_vars = [['TwrBsFyt'], ['OoPDefl1', 'OoPDefl2', 'OoPDefl3']]
fa.ranking_stats = ['max', 'min']
load_ranking, load_ranking_df = fa.load_ranking(stats_df, get_df=True)
load_ranking_df.head()

This is organized for each iteration of `[ranking_vars, ranking_stats]`. The stats are ordered accordingly, and `(stat)_case_idx` refers to the case name index of each load. 

#### Plot load ranking
We can plot the load ranking

In [2]:
windspeed, seed, IECtype, cmw = Processing.get_windspeeds(cm[0], return_df=True)

# Plot bar charts
flag_DLC_name = True
n_ranking     = 10
fig_ext       = '.pdf'
font_size     = 10

clrs = np.array([[127, 60, 141],
                 [17, 165, 121],
                 [57, 105, 172],
                 [242, 183, 1],
                 [231, 63, 116],
                 [128, 186, 90],
                 [230, 131, 16]]) / 256.

for k in range(int(0.5 * len(load_ranking_df.columns))):
    ch_i = int(k*2)
    colors = np.zeros((n_ranking, 3))
    case   = load_ranking_df.columns[ch_i][0]
    channel= load_ranking_df.columns[ch_i][1]
    stat   = load_ranking_df.columns[ch_i][2]
    labels = n_ranking * ['']
    for i in range(n_ranking):
        DLC_class = cm[0][('IEC', 'DLC')][load_ranking_df[case][channel][stat + '_case_idx'][i]]
        if DLC_class == '1.1':
            colors[i,:] = clrs[0]
        elif DLC_class == '1.3':
            colors[i,:] = clrs[1]
        elif DLC_class == '1.4':
            colors[i,:] = clrs[2]
        elif DLC_class == '1.5':
            colors[i,:] = clrs[3]
        elif DLC_class == '5.1':
            colors[i,:] = clrs[4]
        elif DLC_class == '6.1':
            colors[i,:] = clrs[5]
        elif DLC_class == '6.3':
            colors[i,:] = clrs[6]
        else:
            colors[i,:] = clrs[7]

        WS    = windspeed[load_ranking_df[case][channel][stat + '_case_idx'][i]]
        labels[i] = 'DLC ' + DLC_class +' - ' + str(WS) + ' m/s'
        labels_index = load_ranking_df[case][channel][stat + '_case_idx'][0:n_ranking]


    fig, ax = plt.subplots()
    load_ranking_df[case][channel][stat][0:n_ranking].plot.bar(color=colors)
    plt.xlabel('DLC [-]', fontsize=font_size+2, fontweight='bold')
    plt.ylabel(channel + ' ' + uom[k], fontsize=font_size+2, fontweight='bold')
    if flag_DLC_name:
        plt.xticks(np.arange(n_ranking), labels=labels)
    else:
        plt.xticks(np.arange(n_ranking), labels=labels_index)
    plt.subplots_adjust(bottom = 0.5, left = 0.2)
    #fig.savefig(output_folder + 'ranking_' + case + '_' + channel + '_' + stat + fig_ext)
    
plt.show()

NameError: name 'Processing' is not defined

### Wind speed related analysis
We often want to make sense of some batch output data with data binned by windspeed. We can leverage the case-matrix from our output data to figure out the input wind speeds. Of course, `('InflowWind', 'Filename')` must exist in the case matrix. Lets load the wind speeds, save them, and append them to the case matrix as `('InflowWind', 'WindSpeed')`.

In [ ]:
windspeed, seed, IECtype, cmw = Processing.get_windspeeds(cm2, return_df=True)
cmw

### AEP
Now that we know the wind speeds that we were operating at, we can find the AEP. We define the turbine class here, and the cumulative distribution or probability density function 
for the Weibull distribution per IEC 61400 is generated. We can then calculate the AEP. 

If we first want to verify the PDF, we initialize the `power_production` function, define the turbine class, and can plot a CDF for given range:

In [ ]:
pp = Analysis.Power_Production()
pp.turbine_class = 2
Vrange = np.arange(2,26) # Range of wind speeds being considered
weib_prob = pp.prob_WindDist(Vrange,disttype='pdf')
plt.close('all')
plt.plot(Vrange, weib_prob)
plt.grid(True)
plt.xlabel("Wind Speed m/s")
plt.ylabel('Probability')
plt.title('Probability Density Function \n IEC Class 2 Wind Speeds ')
plt.show()


To get the AEP, we need to provide the wind speeds that the simulations were run for, and the corresponding average power results. Internally, in power_production.AEP, the mean power for a given average wind sped is multiplied times the wind speed's probability, then extrapolated to represent yearly production. 

To get the AEP, the process is simple:

In [ ]:
AEP = pp.AEP(stats, windspeed)


##### About the wind speed warning:
Here, we get a warning about the input windspeed array. This is because we passed the complete array output from Processing.get_windspeeds to the AEP function. The input windspeeds to power_production.AEP must satisfy either of the following two conditions:
- each wind speed value corresponds to each each statistic value, so `len(windspeeds) = len(stats_df)`
- each wind speed value corresponds to each run in the case matrix, so `len(windspeeds) = len(cm)`

If the second of these conditions is satisfied, it is assumed that each dataset has the same wind speeds corresponding to each run. So, in this case, the wind speeds corresponding to DLC_1.1 and DLC_1.3 should be the same. 

### Plotting
Finally, we can make some plots. There are a few tools we have at our disposal here. First, we can look at more plots that show our design performance as a function of wind speed. Notably, we can pass the stats dictionary or dataframe to these statistics-related scripts.

Currently, `an_plts.stat_curve()` can plot a "statistics curve" for of two types, a bar or a line graph. 

A bar graph is useful to compare design cases easily:

In [ ]:
plt.close()
an_plts = Analysis.wsPlotting()
an_plts.stat_curve(windspeed, stats, 'RotSpeed', 'bar', names=['DLC1.1', 'DLC1.3'])
plt.show()

A line graph can be useful to show turbulent wind curves. Here we show the means with a first level of errorbars corresponding to standard deviations, and a second level showing minimums and maximums.

In [ ]:
an_plts.stat_curve(windspeed, stats, 'GenPwr', 'line', stat_idx=1, names=['DLC1.3'])
plt.show()

#### Time domain related data
We can also look at our data from the time domain results. 

We can compare any number of channels using the ROSCO toolbox plotting tools. First we'll load two cases to plot together, then plot the time histories.

In [ ]:
#  Load some time domain cases 
filenames = [outfiles[0][7], outfiles[1][7]] # select the 2nd run from each dataset
fast_data = fast_io.load_FAST_out(filenames, tmin=30)
filenames

In [ ]:
# Define the plots we want to make (can be as many or as few channels and plots as you would like...)
cases = {'Baseline': ['Wind1VelX', 'GenPwr', 'BldPitch1', 'GenTq', 'RotSpeed'],
        'Oop' : ['OoPDefl1', 'OoPDefl2', 'OoPDefl13']}

# plot
fast_pl.plot_fast_out(cases, fast_data)
plt.show()

We can additionally do some frequency domain analysis. Here, `spec_cases` is defined by `(channel, run)` where the run index corresponds to the desired plotting index in the loaded fast data.

In [ ]:
spec_cases = [('RootMyb1', 0), ('', 0)]
twrfreq = .0716
fig, ax = fast_pl.plot_spectral(fast_data, spec_cases, 
                                show_RtSpeed=True, RtSpeed_idx=[0],
                                add_freqs=[twrfreq], add_freq_labels=['Tower'],
                                averaging='Welch')
ax.set_title('DLC_1.1')
plt.show()

Finally, we can plot the distribution of any channels from our fast output data

In [ ]:
an_plts = Analysis.wsPlotting()
channels = ['RotSpeed']
caseid = [0,1]
an_plts.distribution(fast_data, channels, caseid, names=['DLC 1.1', 'DLC 1.3'])
plt.show()

# AEP Stuff...

In [ ]:
Vavg = 10

windspeeds = np.array([1.,2.,3.,3.,5.])
ws_set = list(set(windspeeds))
ptemp = np.random.rand(2,5)

In [ ]:
df = pd.DataFrame(ptemp.T)
df['windspeeds'] = windspeeds
df = df.groupby('windspeeds').mean()

In [ ]:
np.trapz(df.T * ws_set, ws_set)

In [ ]:
df['windspeeds']